In [32]:
# REGRESSION MODEL DEEP LEARNING
import os
import gc
import time
import itertools

import matplotlib.pyplot as plt
import pandas as pd
import numpy, textblob, string, random
import tensorflow as tf
from scipy import stats
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from sklearn.externals import joblib
from sklearn.utils import shuffle
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.externals import joblib

from sklearn.metrics import mean_absolute_error
import keras
from keras import backend as K
from keras import models, layers, regularizers
from keras.layers import Dense, Conv2D, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.utils import multi_gpu_model
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
from keras import initializers
from keras.callbacks import EarlyStopping
import tensorflow
from sklearn.metrics import mean_squared_error
from math import sqrt


os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def convert_time(second):
    day = second/86400
    hour = (day - int(day))*24
    minute = (hour - int(hour))*60
    second = round((minute - int(minute))*60,4)
    return(str(int(day)) + ' DAYS: '+ str(int(hour)) + ' HOURS: '+ str(int(minute)) + ' MINUTES: ' + str(second) + ' SECONDS')

In [3]:
def reset_weights(model):
    session = K.get_session()
    for layer in model.layers: 
        if isinstance(layer, keras.engine.network.Network):
            reset_weights(layer)
            continue
        for v in layer.__dict__.values():
            if hasattr(v, 'initializer'):
                v.initializer.run(session=session)

def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))

In [4]:
def build_model(shape):
    model = models.Sequential()
    model.add(layers.Dense(128, activation ='relu', input_shape=(shape,)))
    model.add(layers.Dense(64, activation ='relu'))
    model.add(layers.Dense(32, activation ='relu'))
    model.add(layers.Dense(16, activation ='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam',loss='mse',metrics=['mae'])
    return model

In [6]:
path = 'G:/My Drive/NCSU/Hackathons/HackNC2019/MyPueblo/backend/'
df = pd.read_csv(path + 'data/house_price/train.csv')

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
df = df.fillna(0)
temp = df.apply(pd.to_numeric,errors='coerce').isnull().any()
numeric_error_columns = [df.columns[i] for i in range(len(temp)) if temp[i] == True]
print('Below are the numeric Error Columns. They will be removed. ')
print(numeric_error_columns)
text_df = df[numeric_error_columns]
print(text_df.head())

Below are the numeric Error Columns. They will be removed. 
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
  MSZoning Street Alley LotShape LandContour Utilities LotConfig LandSlope  \
0       RL   Pave     0      Reg         Lvl    AllPub    Inside       Gtl   
1       RL   Pave     0      Reg         Lvl    AllPub       FR2       Gtl   
2       RL   Pave     0      IR1         Lvl    AllPub    Inside       Gtl   
3       RL   Pave     0      IR1         Lvl    

In [13]:
def convert_text_numeric(df):
    headers = df.columns.tolist()
    for each in headers:
        unique = list(set(df[each].values.tolist()))
        actual_list = df[each].values.tolist()
        num_categories = [j for j in range(len(unique))]
        text_num_dict = dict(zip(unique, num_categories))
        numerized_list = [text_num_dict[i] for i in actual_list]
#         print(numerized_list)
        df[each]=numerized_list
    return df

In [28]:
def data_process(pre_all_data):
    '''
    fname should have desc or fing.
    Process data based on fingerprint or descriptor. desc = 'fing' for fingerprint, everything else is descriptors. 
    Return processed data.
    '''
    # convert the dataframe to numpy array
#   pre_all_data = pre_all_data.convert_objects(convert_numeric=True)
    pre_all_data = pre_all_data.fillna(0)
    # all_data_np = pre_all_data.values 

    # drop numeric columns
    temp = pre_all_data.apply(pd.to_numeric,errors='coerce').isnull().any()
    numeric_error_columns = [pre_all_data.columns[i] for i in range(len(temp)) if temp[i] == True]
    print('Below are the numeric Error Columns. Convert to numerical values. ')
    print(numeric_error_columns)
    pre_values_df = pre_all_data.drop(columns=numeric_error_columns)
    text_df = pre_all_data[numeric_error_columns]
    
    # convert textual data frame to numerical data frame
    numeric_df = convert_text_numeric(text_df)
    
    all_df = pd.concat([pre_values_df, numeric_df], axis=1, sort=False)
    # preprocess data for features with original values, normalize columns
    imputer    = SimpleImputer()
    scaler     = preprocessing.MinMaxScaler()
    values_df   = scaler.fit_transform(imputer.fit_transform(all_df)) 

    print("values_df :", values_df.shape)

    return values_df

In [26]:

# numeric_df = convert_text_numeric(text_df)
# print(numeric_df)
    

In [31]:
num_epochs = 100

start_time = time.time()

cv_frame = pd.DataFrame()

path = 'G:/My Drive/NCSU/Hackathons/HackNC2019/MyPueblo/backend/'
df = pd.read_csv(path + 'data/house_price/train.csv')    

df = df.sample(n=len(df), random_state=10)


train_IDs     = df[['Id']]
pre_train_data    = df.drop(columns=['SalePrice', 'Id'])
train_targets  = numpy.concatenate(df[['SalePrice']].values, axis=0).tolist()

train_data = data_process(pre_train_data)

input_nodes = train_data.shape[1]

DNN_model = build_model(input_nodes)
DNN_model.fit(train_data, train_targets,epochs = num_epochs, batch_size=10, verbose=0)
#                       ctrainbacks=[early_stopping_monitor])

predictions = DNN_model.predict(train_data)
predictions_list  = numpy.concatenate(predictions, axis=0).tolist()
model_name = path +'model/house_price.h5'
DNN_model.save(model_name)

rms = sqrt(mean_squared_error(predictions_list, train_targets))
print('RMSE: ', rms)
        
duration = convert_time(time.time()-start_time)
print(duration)

Below are the numeric Error Columns. Convert to numerical values. 
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
values_df : (1460, 79)


C:\Users\kzphy\.conda\envs\ML\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


NameError: name 'sqrt' is not defined

In [9]:
# "Loss"
# plt.plot(history.history['mean_absolute_error'])

In [33]:
rms = sqrt(mean_squared_error(predictions_list, train_targets))
print('RMSE: ', rms)

RMSE:  28639.655998722174
